In [1]:
import os
import json
import pandas as pd

각 알약 조합 별, 모든 알약에 대해 정보가 존재하는 이미지들을 구한다.

In [ ]:
images=sorted(os.listdir('./additional_training_data/train_cleaned'))
annot_root='./additional_training_data/train_annotations'
annots_t=sorted(os.listdir(annot_root)) # 알약 조합 목록
annots_list=[]
images_list=[]
# 각 알약 조합에 대해
for annot_set in annots_t:
    # 약 목록을 구한다.
    pill_list=sorted(os.listdir(os.path.join(annot_root,annot_set)))
    # 첫 약이 가지는 json 파일 목록 (각 이미지에 대한 json 파일 목록)
    base_images=sorted(os.listdir(os.path.join(annot_root,annot_set,pill_list[0])))
    b=len(base_images)
    # 각 약에 대해
    for pill in pill_list:
        # 각 약이 어떤 이미지에 대한 annotation을 갖는지에 대한 list
        imgs=sorted(os.listdir(os.path.join(annot_root,annot_set,pill)))
        # 특정 약은 annotation이 없는 이미지여도 추가
        for img in imgs:
            if img not in base_images:
                base_images.append(img)
    
    # annotation이 하나라도 존재하는 모든 이미지를 추가
    for image in base_images:
        annots_list.append(image[:-5])
# 
for img in images:
    images_list.append(img[:-4])
len(annots_list), annots_list[0], len(images_list),images_list[0]

(8225,
 'K-000250-000573-002483-006192_0_2_0_2_70_000_200',
 8233,
 'K-000250-000573-002483-006192_0_2_0_2_75_000_200')

전체 파일 목록을 추출

In [3]:
images_set=set(images_list)
annots_set=set(annots_list)
# 이미지에 대해, 모든 알약의 annotation이 있는 경우의 이름들을 모음
available_files=sorted(list(images_set.intersection(annots_set)))
len(available_files)

8224

In [4]:
with open('./annots_without_image.txt','w') as f:
    for annot in annots_list:
        if annot not in images_list:
            f.write(str(annot)+'\n')

In [5]:
with open('./images_without_annot.txt','w') as f:
    for image in images_list:
        if image not in annots_list:
            f.write(str(image)+'\n')

In [6]:
for f in available_files:
    print(f)
    break

K-000250-000573-002483-006192_0_2_0_2_75_000_200


이미지 단위로 annotation 파일들을 결합합니다.

In [ ]:
#json 파일 통합

base_path = "./additional_training_data"
ann_root = os.path.join(base_path, "train_annotations")

output_path = os.path.join(base_path, "train_annots")
os.makedirs(output_path, exist_ok=True)

image_id_offset = 1499
annotation_id_offset = 5691
category_set = set()

# 알약 조합 별 폴더 명
for root in sorted(os.listdir(ann_root)):

    # 현재 조합의 알약 종류
    pills=sorted(os.listdir(os.path.join(ann_root,root)))

    # 현재 조합의 이미지 목록
    images_list=[]
    for pill in pills:
        images_list_p=os.listdir(os.path.join(ann_root,root,pill))
        for i in images_list_p:
            if i not in images_list:
                images_list.append(i)

    if len(images_list)==0:
        continue

    # 각 이미지 별 JSON 파일 생성
    for img in images_list:

        images=[]
        annotations=[]
        categories=[]

        # 대응하는 이미지가 없는 annotation이면 continue
        if img[:-5] not in available_files:
            continue
        # 각 알약에 대해서
        for pill in pills:
            # 만약 이번 알약은 해당 이미지에 대한 annotation이 없다면 넘어갑니다.
            if img not in os.listdir(os.path.join(ann_root,root,pill)):
                continue
            p_id=int(pill[2:])
            # 해당 이미지의 json 파일을 엽니다.
            with open(str(os.path.join(ann_root,root,pill,img)), "r", encoding="utf-8") as f:
                pill_data = json.load(f)
                cur_image=pill_data['images']
                cur_image[0]['id']+=image_id_offset
                an=pill_data['annotations']

                # 만약 annotation의 bbox에 대해 4개의 좌표가 주어지지 않았다면
                # 해당 파일은 삭제하고 건너뜁니다.
                if len(an[0]['bbox'])!=4:
                    os.system(f'rm -rf "{os.path.join(ann_root,root,pill,img)}"')
                    continue

                # 이외의 경우 데이터를 추가합니다.
                an[0]['category_id']=(p_id-1)
                an[0]['image_id']=image_id_offset
                an[0]['id']+=annotation_id_offset

                annotation_id_offset+=1
                annotations.extend(an)

                if len(images)==0:
                    images.extend(pill_data["images"])
                    image_id_offset+=1

                cat=pill_data["categories"]
                cat[0]['id']=(p_id-1)
                cat[0]['name']=pill_data["images"][0]['dl_name']
                categories.extend(cat)
        output_coco={
            "images": images,
            "annotations": annotations,
            "categories": categories
        }
        with open(os.path.join(output_path,img), "w", encoding="utf-8") as f:
            json.dump(output_coco, f, ensure_ascii=False, indent=4)

이제 모든 이미지의 json 파일을 결합합니다.

In [8]:
output_all_coco = {
    "images": [],
    "annotations": [],
    "categories": []
}
files = os.listdir(os.path.join(base_path, "train_annots"))

for file in files:
    with open(os.path.join(base_path, "train_annots", file), "r", encoding="utf-8") as f:
        data=json.load(f)
        for image in data['images']:
            output_all_coco['images'].append(image)
        for annot in data['annotations']:
            output_all_coco['annotations'].append(annot)
        for cat in data['categories']:
            output_all_coco['categories'].append(cat)

t=pd.DataFrame(output_all_coco['categories'])
t=t.drop_duplicates()
output_all_coco['categories']=t.to_dict(orient='records')

save_path = os.path.join(base_path, "train.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(output_all_coco, f, ensure_ascii=False, indent=4)

print("train.json 생성됨:", save_path)

train.json 생성됨: ./additional_training_data/train.json


In [9]:
len(output_all_coco['categories'])

72

In [10]:
len(output_all_coco['images'])

8233